# Load dataset

In [1]:
import numpy as np

In [2]:
xy = np.load("openface_and_labels_by_frame.npz")
X, y = xy["x"], xy["y"]

In [3]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (604901, 713)
y shape: (604901, 7)


In [4]:
print(X[1].shape)

(713,)


# Exploration of data

In [5]:
X_sample = X[np.random.choice(len(X))]
y_sample = y[np.random.choice(len(y))]
print("random X min:", np.min(X_sample))
print("random X max:", np.max(X_sample))
print("random y min:", np.min(y_sample))
print("random y max:", np.max(y_sample))

random X min: -115.075
random X max: 797.1938
random y min: 0.0
random y max: 3.0


In [6]:
print("x min:", np.min(X))
print("x max:", np.max(X))

x min: -34065332.0
x max: 48234530.0


In [7]:
def summarize_y(y):
    for i in range(y.shape[1]):
        print("min y, index", i, ":", np.min(y[:,i]))
        print("max y, index", i, ":", np.max(y[:,i]), "\n")

    print("y min:", np.min(y[:,4]))
    print("y max:", np.max(y))
    print("y mean:", np.mean(y))
    print("y std:", np.std(y), "\n")

In [8]:
summarize_y(y)

min y, index 0 : -3.0
max y, index 0 : 3.0 

min y, index 1 : 0.0
max y, index 1 : 3.0 

min y, index 2 : 0.0
max y, index 2 : 3.0 

min y, index 3 : 0.0
max y, index 3 : 3.0 

min y, index 4 : 0.0
max y, index 4 : 3.0 

min y, index 5 : 0.0
max y, index 5 : 3.0 

min y, index 6 : 0.0
max y, index 6 : 1.6666666 

y min: 0.0
y max: 3.0
y mean: 0.17889453
y std: 0.56088483 



Note: per the paper, the integral [-3, 3] for the target values, as implied above (min/max) represents sentiment, with +3 being highly positive, -3 highly negative, 0 neutral. the mean value is also consistent with the distribution shown in the paper (pp 2240 fig 2), which shows that most annotations are neutral or weakly positive (0-1). do we need to balance the classes at some point? **note that the first index (i=0) of any given target array represents this integral.

The paper mentions a second integral [0,3] which measures the presence of emotion (0 = no presence, 3 = highest presence). It looks like indices 1-5 of the target arrays are of this integral.

In [9]:
np.unique(y[:,1])

array([0.        , 0.16666667, 0.33333334, 0.5       , 0.6666667 ,
       0.8333333 , 1.        , 1.1666666 , 1.3333334 , 1.6666666 ,
       2.        , 2.1666667 , 2.3333333 , 2.6666667 , 3.        ],
      dtype=float32)

Note: it looks like the values are specifically sixths (1/6) between the intervals I specified above for each of the 7 labels. once I was sure of this, I went ahead and converted them to integer representations.

In [10]:
y_encoded = y * 6
np.unique(y_encoded)

array([-18., -16., -14., -12., -10.,  -8.,  -6.,  -4.,  -3.,  -2.,  -1.,
         0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  18.], dtype=float32)

In [11]:
y_encoded = np.round(y_encoded).astype(np.int64)

In [12]:
summarize_y(y_encoded)

min y, index 0 : -18
max y, index 0 : 18 

min y, index 1 : 0
max y, index 1 : 18 

min y, index 2 : 0
max y, index 2 : 18 

min y, index 3 : 0
max y, index 3 : 18 

min y, index 4 : 0
max y, index 4 : 18 

min y, index 5 : 0
max y, index 5 : 18 

min y, index 6 : 0
max y, index 6 : 10 

y min: 0
y max: 18
y mean: 1.0733673774716854
y std: 3.365308890115125 



# Split data

We can decide later exactly how much of the data ought to be used, but for the sake of testing things for now, I've truncated the dataset to something more manageable from a time perspective (sample a number (n_samples * m_percent) of random indices and extract those indices from the features/targets). I can perform a larger-in-scale training session over break on my GPU desktop (hopefully without interruption due to hardware limitations) once we're satisfied with the model parameterization

In [73]:
def truncate_data(X, y, n_samples):
    random_i = np.random.choice(len(X), size = n_samples)
    X_trunc = X[random_i]
    y_trunc = y[random_i]
    return X_trunc, y_trunc

In [74]:
X_truncated, y_truncated_emotion = truncate_data(X, y_encoded[:,1:], int(X.shape[0] * 0.5))

In [75]:
y_truncated_sentiment = truncate_data(X, y_encoded[:,0], int(X.shape[0] * 0.5))[1]

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_truncated, y_truncated_emotion, test_size = 0.2)

print("Train:", X_train.shape[0])
print("Test:", X_test.shape[0])
print("y shape:", y_train.shape)

Train: 241960
Test: 60490
y shape: (241960, 6)


In [77]:
# Make y one-hot
y_train = tf.one_hot(y_train, depth=4, axis=-1)
y_train = tf.reshape(y_train, (y_train.shape[0], -1))
y_train.shape

TensorShape([241960, 24])

# Model

## Predicting presence of emotions

In [132]:
import tensorflow as tf

# Using functional model to make there be one-hot encodings for each emotion
input = tf.keras.Input(shape=(X_train.shape[1],))

x = tf.keras.layers.Dense(512, activation='elu', kernel_regularizer="l1_l2")(input)
x = tf.keras.layers.Dense(256, activation='elu', kernel_regularizer="l1_l2")(x)
x = tf.keras.layers.Dense(128, activation='elu', kernel_regularizer="l1_l2")(x)
x = tf.keras.layers.Dense(64, activation='elu', kernel_regularizer="l1_l2")(x)
x = tf.keras.layers.Dense(6*4, kernel_regularizer="l1_l2")(x)
x = tf.reshape(x, (-1, 6, 4))
x = tf.keras.activations.softmax(x, axis=-1)
x = tf.reshape(x, (-1, 6 * 4))

emotion_model = tf.keras.models.Model(input, x)
emotion_model.summary()

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 713)]             0         
                                                                 
 dense_107 (Dense)           (None, 512)               365568    
                                                                 
 dense_108 (Dense)           (None, 256)               131328    
                                                                 
 dense_109 (Dense)           (None, 128)               32896     
                                                                 
 dense_110 (Dense)           (None, 64)                8256      
                                                                 
 dense_111 (Dense)           (None, 24)                1560      
                                                                 
 tf.reshape_29 (TFOpLambda)  (None, 6, 4)              0  

In [133]:
mse_loss = tf.keras.losses.MeanSquaredError()
cce_loss = tf.keras.losses.CategoricalCrossentropy()

In [134]:
emotion_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
  loss=mse_loss,
  metrics=["binary_accuracy", tf.keras.metrics.Precision()])

In [135]:
emotion_model.fit(X_train, y_train, epochs=10, validation_split=0.2, shuffle=True, batch_size=64)

Epoch 1/10
3025/3025 [==============================] - 29s 9ms/step - loss: 22.3795 - binary_accuracy: 0.8837 - precision_20: 0.7262 - val_loss: 0.1783 - val_binary_accuracy: 0.9175 - val_precision_20: 0.8054
Epoch 2/10
3025/3025 [==============================] - 27s 9ms/step - loss: 0.1593 - binary_accuracy: 0.9172 - precision_20: 0.8079 - val_loss: 0.1516 - val_binary_accuracy: 0.9185 - val_precision_20: 0.8160
Epoch 3/10
3025/3025 [==============================] - 26s 9ms/step - loss: 0.1474 - binary_accuracy: 0.9177 - precision_20: 0.8130 - val_loss: 0.1449 - val_binary_accuracy: 0.9194 - val_precision_20: 0.8227
Epoch 4/10
3025/3025 [==============================] - 26s 9ms/step - loss: 0.1430 - binary_accuracy: 0.9169 - precision_20: 0.7969 - val_loss: 0.1393 - val_binary_accuracy: 0.9140 - val_precision_20: 0.7609
Epoch 5/10
3025/3025 [==============================] - 26s 9ms/step - loss: 0.1271 - binary_accuracy: 0.9136 - precision_20: 0.7600 - val_loss: 0.1221 - val_binar

KeyboardInterrupt: 

In [136]:
emotion_model.save_weights("em1/em1_checkpoint")

In [143]:
# Evaluate
y_pred = tf.reshape(emotion_model.predict(X_test), (-1, 6, 4))
y_test = tf.one_hot(y_test, depth=4, axis=-1)

for i in range(6):
    acc = tf.keras.metrics.categorical_accuracy(
        y_test[:, i, :], y_pred[:, i, :]
    )
    print("Emotion %d: %.4f Accuracy" % (i, tf.math.reduce_mean(acc)))


1891/1891 [==============================] - 5s 3ms/step
Emotion 0: 0.8027 Accuracy
Emotion 1: 0.8702 Accuracy
Emotion 2: 0.9132 Accuracy
Emotion 3: 0.9223 Accuracy
Emotion 4: 0.9162 Accuracy
Emotion 5: 0.9403 Accuracy


## Sentiment prediction

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X_truncated, y_truncated_sentiment, test_size = 0.2)

print("Train:", X_train.shape[0])
print("Test:", X_test.shape[0])
print("y shape:", y_train.shape)

y_train = tf.one_hot(y_train, depth=7, axis=-1)
print(y_train.shape)

Train: 241960
Test: 60490
y shape: (241960,)
(241960, 7)
